In [ ]:
import json
import os
from invariant.analyzer import Policy
from invariant.analyzer.traces import *

import nest_asyncio
nest_asyncio.apply()

In [ ]:
!git clone https://github.com/swe-bench/experiments.git
data_folder = 'experiments/evaluation/test/20240402_sweagent_gpt4/trajs'

In [ ]:
def print_trace(trace, abbreviate=True):
    """Utility function to print the trace"""
    for t in trace:
        if t.get("role", None) in ["system", "user", "assistant"]:
            type, content = t["role"], t["content"]
            print(f"{type}: {content}")
        elif t.get("role", None) == "tool":
            content = "<...>" if abbreviate else t["content"]
            print("tool_output: ", content)
        else:
            args = "<...>" if abbreviate else t["function"]["arguments"]
            print("tool_call: ", t["function"]["name"], args)

In [ ]:
def transform_trace(trajectory):
    """Utility to transform the trajectory to trace that can be analyzed by Invariant"""
    inv_traj = []
    for idx, el in enumerate(trajectory):
        action = el["action"]
        action_name = action[:action.find(" ")]
        action_params = action[action.find(" ")+1:]

        if action_name == "edit":
            code = action[action.find("\n"):action.rfind("end_of_edit")]
            loc = action_params[:action_params.find("\n")]
            inv_traj.append(tool_call(idx, "edit", {"code": code, "loc": loc}))
        else:
            inv_traj.append(tool_call(idx, action_name, {"arg": action_params}))

        observation = el["observation"]
        inv_traj.append(tool(idx, observation))
    return inv_traj

In [ ]:
# We load all the tracea
traces = []
for _, _, files in os.walk(data_folder):
    for idx, file in enumerate(files):
        file_path = os.path.join(data_folder, file)
        with open(file_path, 'r') as fin:
            content = fin.read()
            input_data = json.loads(content)

            trajectory = input_data["trajectory"]
            traces.append(transform_trace(trajectory))
print(f"Loaded {len(traces)} traces")

We consider a policy that raises a violation if there have been two tool calls such that the first call introduces a potential security vulnerability via pickle, and one of the next calls runs the python code.

In [ ]:
policy = Policy.from_string("""
    from invariant.detectors import pii, semgrep, CodeIssue, secrets
                                
    raise PolicyViolation("found unsafe code: ", issue) if:
        (call1: ToolCall) -> (call2: ToolCall)
        call1.function.name == "edit"
        (issue: CodeIssue) in semgrep(call1.function.arguments["code"])
        call2.function.name == "python"
        "pickle" in issue.description
    """)

In [ ]:
bad_idx = []
for idx, trace in enumerate(traces):
    res = policy.analyze(trace)
    if len(res.errors) > 0:
        bad_idx.append((idx, res))
    if (idx+1) % 100 == 0:
        print(f"bad traces: {len(bad_idx)}/{idx+1}")        